In [1]:
import pandas as pd

# Read the CSV file directly
FILEPATH = 'dynamic_superstore - Data.csv'
df = pd.read_csv(FILEPATH)

# Check the first few rows or columns to inspect data
print(df.head())  # Display first 5 rows
print(df.columns) # Show column names

   Row_Id  Order Date  Year Month   Ship Date  Delivery Time       Ship Mode  \
0       1   11/9/2013  2013   Nov  11/12/2013              3    Second Class   
1       2   11/9/2013  2013   Nov  11/12/2013              3    Second Class   
2       3   6/13/2013  2013   Jun   6/17/2013              4    Second Class   
3       4  10/11/2012  2012   Oct  10/18/2012              7  Standard Class   
4       5  10/11/2012  2012   Oct  10/18/2012              7  Standard Class   

     Customer Name    Segment        Country             City       State  \
0      Claire Gute   Consumer  United States        Henderson    Kentucky   
1      Claire Gute   Consumer  United States        Henderson    Kentucky   
2  Darrin Van Huff  Corporate  United States      Los Angeles  California   
3   Sean O'Donnell   Consumer  United States  Fort Lauderdale     Florida   
4   Sean O'Donnell   Consumer  United States  Fort Lauderdale     Florida   

  Region         Category Sub-Category  \
0  South      

In [2]:
# Display the first few rows of the dataset to confirm its structure
print(df.head())

# Check for missing values and data types to ensure readiness for analysis
print('\
Missing Values:')
print(df.isnull().sum())

print('\
Data Types:')
print(df.dtypes)

   Row_Id  Order Date  Year Month   Ship Date  Delivery Time       Ship Mode  \
0       1   11/9/2013  2013   Nov  11/12/2013              3    Second Class   
1       2   11/9/2013  2013   Nov  11/12/2013              3    Second Class   
2       3   6/13/2013  2013   Jun   6/17/2013              4    Second Class   
3       4  10/11/2012  2012   Oct  10/18/2012              7  Standard Class   
4       5  10/11/2012  2012   Oct  10/18/2012              7  Standard Class   

     Customer Name    Segment        Country             City       State  \
0      Claire Gute   Consumer  United States        Henderson    Kentucky   
1      Claire Gute   Consumer  United States        Henderson    Kentucky   
2  Darrin Van Huff  Corporate  United States      Los Angeles  California   
3   Sean O'Donnell   Consumer  United States  Fort Lauderdale     Florida   
4   Sean O'Donnell   Consumer  United States  Fort Lauderdale     Florida   

  Region         Category Sub-Category  \
0  South      

In [3]:
# Check for duplicates and clean the data to ensure accuracy
df = df.drop_duplicates()  # Assign back instead of using inplace=True

# Handle missing values in the 'Country' column by filling with 'Unknown'
df['Country'] = df['Country'].fillna('Unknown')  # No inplace=True

# Confirm the cleaning process
print('Data cleaned:')
print(df.isnull().sum())

Data cleaned:
Row_Id           0
Order Date       0
Year             0
Month            0
Ship Date        0
Delivery Time    0
Ship Mode        0
Customer Name    0
Segment          0
Country          0
City             0
State            0
Region           0
Category         0
Sub-Category     0
Product Name     0
Sales            0
Quantity         0
Discount         0
Profit           0
dtype: int64


In [4]:
# Set up the visual style for all plots
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# Load and clean the data
FILEPATH = 'dynamic_superstore - Data.csv'
df = pd.read_csv(FILEPATH)

# Clean 'Sales' and 'Profit' columns (if needed)
for col in ['Profit', 'Sales']:
    df[col] = df[col].replace({r'\$': '', ',': ''}, regex=True)  # Use raw string
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Set dark theme style
plt.style.use('dark_background')

# Rest of your existing code...

# 1. Overall Sales Trend
df['Order Date'] = pd.to_datetime(df['Order Date'])
monthly_sales = df.groupby(df['Order Date'].dt.to_period('M')).agg({
    'Sales': 'sum',
    'Profit': 'sum'
}).reset_index()
monthly_sales['Order Date'] = monthly_sales['Order Date'].astype(str)

# Create an interactive sales trend plot
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=monthly_sales['Order Date'],
    y=monthly_sales['Sales'],
    mode='lines',
    name='Sales',
    line=dict(color='#00ff88', width=2)
))
fig.add_trace(go.Scatter(
    x=monthly_sales['Order Date'],
    y=monthly_sales['Profit'],
    mode='lines',
    name='Profit',
    line=dict(color='#ff6b6b', width=2)
))

fig.update_layout(
    template='plotly_dark',
    paper_bgcolor='rgba(0,0,0,0.9)',
    plot_bgcolor='rgba(0,0,0,0.8)',
    title={
        'text': 'Sales and Profit Trends Over Time',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(size=24, color='white')
    },
    xaxis_title='Date',
    yaxis_title='Amount ($)',
    font=dict(family='Arial', size=12, color='white'),
    margin=dict(t=100, l=50, r=50, b=50),
    showlegend=True,
    legend=dict(
        bgcolor='rgba(0,0,0,0.5)',
        bordercolor='rgba(255,255,255,0.2)',
        borderwidth=1
    )
)

# Save the interactive plot
fig.write_html('sales_trend.html')

print('Sales trend dashboard has been generated and saved as sales_trend.html')

Sales trend dashboard has been generated and saved as sales_trend.html


In [5]:
# 2. Category Performance Analysis
# Clean and convert 'Profit' and 'Sales' to numeric
for col in ['Profit', 'Sales']:
    # Remove commas and dollar signs (use raw string for regex)
    df[col] = df[col].replace({r'\$': '', ',': ''}, regex=True)
    # Convert to numeric (errors='coerce' turns invalid values to NaN)
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Re-calculate category_performance with cleaned data
category_performance = df.groupby('Category').agg({
    'Sales': 'sum',
    'Profit': 'sum',
    'Quantity': 'sum'
}).reset_index()

# Create a subplot with category performance metrics
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Sales by Category', 'Profit by Category', 
                   'Quantity Sold by Category', 'Profit Margin by Category'),
    specs=[[{'type': 'pie'}, {'type': 'pie'}],
           [{'type': 'pie'}, {'type': 'pie'}]]
)

colors = ['#00ff88', '#ff6b6b', '#45b3e7']

# Sales
fig.add_trace(
    go.Pie(labels=category_performance['Category'], 
           values=category_performance['Sales'],
           marker=dict(colors=colors)),
    row=1, col=1
)

# Profit
fig.add_trace(
    go.Pie(labels=category_performance['Category'], 
           values=category_performance['Profit'],
           marker=dict(colors=colors)),
    row=1, col=2
)

# Quantity
fig.add_trace(
    go.Pie(labels=category_performance['Category'], 
           values=category_performance['Quantity'],
           marker=dict(colors=colors)),
    row=2, col=1
)

# Profit Margin
category_performance['Profit_Margin'] = (category_performance['Profit'] / category_performance['Sales']) * 100
fig.add_trace(
    go.Pie(labels=category_performance['Category'], 
           values=category_performance['Profit_Margin'],
           marker=dict(colors=colors)),
    row=2, col=2
)

fig.update_layout(
    template='plotly_dark',
    paper_bgcolor='rgba(0,0,0,0.9)',
    plot_bgcolor='rgba(0,0,0,0.8)',
    title={
        'text': 'Category Performance Analysis',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(size=24, color='white')
    },
    showlegend=True,
    legend=dict(
        bgcolor='rgba(0,0,0,0.5)',
        bordercolor='rgba(255,255,255,0.2)',
        borderwidth=1
    ),
    height=800
)

# Save the category performance dashboard
fig.write_html('category_performance.html')

print('Category performance dashboard has been generated and saved as category_performance.html')

Category performance dashboard has been generated and saved as category_performance.html


In [6]:
# 3. Regional Performance and Customer Segments
# Create a combined dashboard for regional and customer segment analysis

# Regional Analysis
regional_performance = df.groupby('Region').agg({
    'Sales': 'sum',
    'Profit': 'sum',
    'Quantity': 'sum'
}).reset_index()

# Customer Segment Analysis
segment_performance = df.groupby('Segment').agg({
    'Sales': 'sum',
    'Profit': 'sum',
    'Quantity': 'sum'
}).reset_index()

# Create the subplot figure
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Regional Sales Distribution', 'Regional Profit Distribution',
                   'Customer Segment Sales', 'Customer Segment Profit'),
    specs=[[{'type': 'bar'}, {'type': 'bar'}],
           [{'type': 'pie'}, {'type': 'pie'}]]
)

# Regional Sales
fig.add_trace(
    go.Bar(x=regional_performance['Region'],
           y=regional_performance['Sales'],
           name='Sales',
           marker_color='#00ff88'),
    row=1, col=1
)

# Regional Profit
fig.add_trace(
    go.Bar(x=regional_performance['Region'],
           y=regional_performance['Profit'],
           name='Profit',
           marker_color='#ff6b6b'),
    row=1, col=2
)

# Segment Sales
fig.add_trace(
    go.Pie(labels=segment_performance['Segment'],
           values=segment_performance['Sales'],
           name='Sales by Segment',
           marker=dict(colors=['#00ff88', '#ff6b6b', '#45b3e7'])),
    row=2, col=1
)

# Segment Profit
fig.add_trace(
    go.Pie(labels=segment_performance['Segment'],
           values=segment_performance['Profit'],
           name='Profit by Segment',
           marker=dict(colors=['#00ff88', '#ff6b6b', '#45b3e7'])),
    row=2, col=2
)

fig.update_layout(
    template='plotly_dark',
    paper_bgcolor='rgba(0,0,0,0.9)',
    plot_bgcolor='rgba(0,0,0,0.8)',
    title={
        'text': 'Regional and Customer Segment Analysis',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(size=24, color='white')
    },
    showlegend=True,
    legend=dict(
        bgcolor='rgba(0,0,0,0.5)',
        bordercolor='rgba(255,255,255,0.2)',
        borderwidth=1
    ),
    height=800
)

# Save the regional and segment analysis dashboard
fig.write_html('regional_segment_analysis.html')

print('Regional and segment analysis dashboard has been generated and saved as regional_segment_analysis.html')

Regional and segment analysis dashboard has been generated and saved as regional_segment_analysis.html


In [7]:
# 4. Marketing Insights Dashboard

# Calculate key metrics
avg_order_value = df.groupby('Order Date')['Sales'].mean()
customer_lifetime_value = df.groupby('Customer Name')['Sales'].sum().mean()
repeat_customers = df.groupby('Customer Name').size()
repeat_customer_rate = (repeat_customers[repeat_customers > 1].count() / repeat_customers.count()) * 100

# Product Performance
product_performance = df.groupby('Sub-Category').agg({
    'Sales': 'sum',
    'Quantity': 'sum',
    'Profit': 'sum'
}).reset_index()
product_performance['Profit_Margin'] = (product_performance['Profit'] / product_performance['Sales']) * 100

# Create the marketing insights dashboard
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Top 10 Products by Sales', 'Product Categories Profit Margin',
                   'Customer Segment Distribution', 'Ship Mode Analysis'),
    specs=[[{'type': 'bar'}, {'type': 'bar'}],
           [{'type': 'pie'}, {'type': 'pie'}]]
)

# Top 10 Products
top_products = product_performance.nlargest(10, 'Sales')
fig.add_trace(
    go.Bar(x=top_products['Sub-Category'],
           y=top_products['Sales'],
           marker_color='#00ff88'),
    row=1, col=1
)

# Profit Margins
fig.add_trace(
    go.Bar(x=product_performance['Sub-Category'],
           y=product_performance['Profit_Margin'],
           marker_color='#ff6b6b'),
    row=1, col=2
)

# Customer Segments
segment_dist = df['Segment'].value_counts()
fig.add_trace(
    go.Pie(labels=segment_dist.index,
           values=segment_dist.values,
           marker=dict(colors=['#00ff88', '#ff6b6b', '#45b3e7'])),
    row=2, col=1
)

# Shipping Analysis
shipping_dist = df['Ship Mode'].value_counts()
fig.add_trace(
    go.Pie(labels=shipping_dist.index,
           values=shipping_dist.values,
           marker=dict(colors=['#00ff88', '#ff6b6b', '#45b3e7', '#9b59b6'])),
    row=2, col=2
)

fig.update_layout(
    template='plotly_dark',
    paper_bgcolor='rgba(0,0,0,0.9)',
    plot_bgcolor='rgba(0,0,0,0.8)',
    title={
        'text': 'Marketing Insights Dashboard',
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(size=24, color='white')
    },
    showlegend=True,
    legend=dict(
        bgcolor='rgba(0,0,0,0.5)',
        bordercolor='rgba(255,255,255,0.2)',
        borderwidth=1
    ),
    height=800
)

# Save the marketing insights dashboard
fig.write_html('marketing_insights.html')

# Print key metrics
print('Key Marketing Metrics:')
print(f'Average Order Value: ${avg_order_value.mean():.2f}')
print(f'Customer Lifetime Value: ${customer_lifetime_value:.2f}')
print(f'Repeat Customer Rate: {repeat_customer_rate:.1f}%')

Key Marketing Metrics:
Average Order Value: $225.51
Customer Lifetime Value: $2896.85
Repeat Customer Rate: 99.4%
